In [1]:
# -*- coding: utf-8 -*-
import os
import datetime as dt

import pandas as pd
from tqdm import tqdm

from pymongo import MongoClient
# from s_lib import setup_mongo, setup_mecab

In [2]:
def daterange(_start, _end):
    for n in range((_end - _start + timedelta(days=1)).days):
        yield _start + timedelta(n)

def open_relation_words(pref, flag):
    dirpath = "./"
    file_name = pref + "_"+ flag + "_soa.txt"
    file_path = dirpath + file_name

    df = pd.read_csv(file_path, names=('word', 'soa'), header=None)
    # 関連度が正でないものは捨てる
    relation_words = df[df.soa > 0].word.tolist()
    return relation_words

def count_twis(db, flag, relation_words):
    rates = range(10, 101, 10)

    JST = dt.timezone(dt.timedelta(hours=9))
    correct = {
    "tk": [dt.datetime(2014, 11, 25, tzinfo=JST), dt.datetime(2014, 12, 18, tzinfo=JST)],
    "hk": [dt.datetime(2014, 10, 30, tzinfo=JST), dt.datetime(2014, 11, 3, tzinfo=JST)],
    "is": [dt.datetime(2014, 11, 13, tzinfo=JST), dt.datetime(2014, 11, 29, tzinfo=JST)]
    }

In [3]:
def main():
    result_dir = '/now24/naruse/out/2015/count/'

    flags = ["icho", "kaede", "sonota", "koyo"]
    prefectures = ["tk", "hk", "is"]

    for pref in  prefectures:
        result_dir = result_dir + pref + "/"

        for flag in flags:
            relation_words = open_relation_words(pref, flag)
            # count_list = count_twis(db, flag, relation_words)
            for i in range(0, 10):
                rate = i*10+10
                filename = flag + "_soa" + str(rate) + "count" + ".tsv"
                filepath = result_dir + filename

main()

In [12]:
rates = range(10, 101, 10)
len = 2001
for rate in rates:
        limit = round(len * (rate / 100))

a = list(range(100))
a[0:50]

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49]

In [ ]:


def count(db, flag):
    # 日毎に、与えられたflagフィールドが1のdocumentをカウントする。
    # 返すリストの要素は"day'\t'month'\t'count"の文字列
    all_day_list = []


    start = datetime(2015, 2, 17, tzinfo=timezone(timedelta(hours=9)))
    end = datetime(2015, 12, 31, tzinfo=timezone(timedelta(hours=9)))

    for date in daterange(start, end):
        today = date.isoformat()
        next_day = (date + timedelta(days=1)).isoformat()


        if flag == "total":
            where = {
                'created_at_iso': {
                    '$gte': today,
                    '$lt': next_day
                }
            }
        elif flag == "koyo":
            where = {
                '$or': [ {'icho': 1}, {'kaede': 1}, {'others': 1} ],
                'created_at_iso': {
                    '$gte': today,
                    '$lt': next_day
                }
            }
        else:
            where = {
                flag : 1,
                'created_at_iso': {
                    '$gte': today,
                    '$lt': next_day
                }
            }

        month = str(date.month).zfill(2)
        day = str(date.day).zfill(2)

        col = db['2015-' + month]
        one_day_count = col.find(where).count()

        one_day = '\t'.join([month, day, str(one_day_count)])
        all_day_list.append(one_day)

    return all_day_list

def main():
    result_dir = '/now24/naruse/out/2015/count/'

    db_tk = setup_mongo('2015_tk_twi')
    db_hk = setup_mongo('2015_hk_twi_1208')
    db_is = setup_mongo('2015_is_twi')

    flags = ["icho", "kaede", "sonota", "koyo", "total"]
    dbs = [db_tk, db_hk, db_is]
    prefecture_list = ["tk", "hk", "is"]

    for pref, db in zip(prefecture_list, dbs):
        result_dir = result_dir + pref + "/"

        for flag in tqdm(pref, flags):
            relation_words = open_relation_words(flag, pref)
            count_list = count(db, flag, relation_words)
            for i in tqdm(range(0, 10)):
                rate = i*10+10
                filename = flag + "_soa" + rate + "count" + ".tsv"
                filepath = result_dir + filename
                os.makedirs(result_dir, exist_ok=True)
                with open(filepath, "w") as f:
                    f.write("\n".join(m_d_count_list[rate/10 - 1]))

main()
